In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, MaxPool2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_images = np.load('drive/MyDrive/Colab Notebooks/train_images_daq_pi_change7.npy')
train_labels = np.load('drive/MyDrive/Colab Notebooks/train_labels_daq_pi_change7.npy')
test_images = np.load('drive/MyDrive/Colab Notebooks/test_images_kosdaq_ma.npy')
test_labels = np.load('drive/MyDrive/Colab Notebooks/test_labels_kosdaq_ma.npy')

In [8]:
from sklearn.model_selection import train_test_split

train_images = np.array(train_images).reshape(-1, 96, 96, 3) / 255.0
test_images = np.array(test_images).reshape(-1, 96, 96, 3) / 255.0

# train_images, test_images, train_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.2)

train_labels_one_hot = np.eye(2)[train_labels]
test_labels_one_hot = np.eye(2)[test_labels]

In [4]:
model = Sequential()

# Convolutional Block (Conv-Conv-Pool-Dropout)
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(96, 96, 3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Block (Conv-Conv-Pool-Dropout)
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) # change this line
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [5]:
import keras

def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 3:
        lrate = 0.0005
    if epoch > 5:
        lrate = 0.0001
    if epoch > 10:
        lrate = 0.00005

    return lrate

checkpoint = keras.callbacks.ModelCheckpoint('best_cnn_model.h5', save_best_only=True, monitor='val_acc')
# N번의 연속적인 epoch 동안 개선이 없을 때 학습이 중단되도록 함
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=30, restore_best_weights=True)

In [6]:
from keras.callbacks import LearningRateScheduler

history = model.fit(train_images, train_labels_one_hot, batch_size=32, epochs=50, validation_data=(test_images, test_labels_one_hot),
                    callbacks=[checkpoint, LearningRateScheduler(lr_schedule)])

Epoch 1/50
313/313 [==============================] - 56s 151ms/step - loss: 2.1244 - acc: 0.5081 - val_loss: 0.6935 - val_acc: 0.5000 - lr: 0.0010
Epoch 2/50
313/313 [==============================] - 30s 95ms/step - loss: 0.6934 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.4999 - lr: 0.0010
Epoch 3/50
313/313 [==============================] - 29s 94ms/step - loss: 0.6941 - acc: 0.4988 - val_loss: 0.6935 - val_acc: 0.4986 - lr: 0.0010
Epoch 4/50
313/313 [==============================] - 36s 115ms/step - loss: 0.6930 - acc: 0.5046 - val_loss: 0.6932 - val_acc: 0.5047 - lr: 0.0010
Epoch 5/50
313/313 [==============================] - 35s 113ms/step - loss: 0.6922 - acc: 0.5182 - val_loss: 0.6930 - val_acc: 0.5073 - lr: 5.0000e-04
Epoch 6/50
313/313 [==============================] - 40s 127ms/step - loss: 0.6919 - acc: 0.5176 - val_loss: 0.6928 - val_acc: 0.5095 - lr: 5.0000e-04
Epoch 7/50
313/313 [==============================] - 35s 110ms/step - loss: 0.6902 - acc: 0.5216 - val_lo

In [1]:
model.save('model.h5')

NameError: ignored

In [3]:
from keras.models import load_model

model = load_model('model.h5')

In [ ]:
import matplotlib.pyplot as plt

plt.figure()

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'], label='acc')
plt.plot(history.history['val_acc'], label='val_acc')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')

# 손실 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='best')

plt.show()

In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels_one_hot, verbose=2)

313/313 - 14s - loss: 0.7774 - acc: 0.5100 - 14s/epoch - 44ms/step


In [10]:
# 테스트 데이터셋에 대한 예측 생성
pred_probs = model.predict(test_images[:100])

print(pred_probs)

4/4 [==============================] - 1s 117ms/step
[[0.4894575  0.5105425 ]
 [0.7097473  0.2902527 ]
 [0.3436195  0.65638053]
 [0.55322033 0.4467797 ]
 [0.4519771  0.5480229 ]
 [0.37773326 0.6222667 ]
 [0.2558756  0.7441244 ]
 [0.5433163  0.4566837 ]
 [0.509894   0.490106  ]
 [0.32955703 0.670443  ]
 [0.04710062 0.9528994 ]
 [0.5173402  0.48265985]
 [0.49411744 0.5058826 ]
 [0.01350198 0.986498  ]
 [0.55837923 0.44162083]
 [0.631706   0.36829397]
 [0.47994617 0.5200538 ]
 [0.5005365  0.4994635 ]
 [0.7097529  0.29024705]
 [0.64345515 0.35654482]
 [0.66527444 0.3347255 ]
 [0.05523936 0.9447606 ]
 [0.5090976  0.49090248]
 [0.26116922 0.7388308 ]
 [0.11551256 0.88448745]
 [0.545662   0.45433798]
 [0.23600148 0.76399857]
 [0.49000466 0.50999534]
 [0.37173647 0.62826353]
 [0.34612262 0.6538773 ]
 [0.09554455 0.9044554 ]
 [0.31230935 0.6876906 ]
 [0.34648606 0.6535139 ]
 [0.6544299  0.34557015]
 [0.10559447 0.89440554]
 [0.62361    0.37638998]
 [0.3177457  0.6822543 ]
 [0.4981132  0.5018868

In [ ]:
model.save('model.h5')